In [1]:
# import
import csv
import math
import random
import sys
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

from sklearn.neighbors import NearestNeighbors

import torch
import torch.nn as nn
import torch.nn.functional as F

from IPython.display import clear_output

%matplotlib inline

random.seed

<bound method Random.seed of <random.Random object at 0x103886218>>

In [2]:
def IoW(gt, coor, radius_gt, radius):
    ### IoW cross section calculation
#     print("s-1")
    cross_list_0 = [(gt[0]-radius_gt, 0), (gt[0]+radius_gt, 0)]
    cross_list_1 = [(gt[1]-radius_gt, 0), (gt[1]+radius_gt, 0)]
#     print("s-2")
    cross_list_0.append((coor[0]-radius, 1))
    cross_list_0.append((coor[0]+radius, 1))
    cross_list_1.append((coor[1]-radius, 1))
    cross_list_1.append((coor[1]+radius, 1))
#     print("s-3")                 
    cross_list_0.sort(key = lambda x : x[0])
    cross_list_1.sort(key = lambda x : x[0])
#     print("s-4") 
    if (cross_list_0[0][1] != cross_list_0[1][1] and cross_list_1[0][1] != cross_list_1[1][1]):
        return (cross_list_0[2][0] - cross_list_0[1][0]) * (cross_list_1[2][0] - cross_list_1[1][0]) / ((radius*2)**2)
    else:
        return 0.
    
    
def distance_progress(coor_gt, coor_cur, coor_next):
    dis_cur = math.sqrt((coor_gt[0]-coor_cur[0])**2 + (coor_gt[1]-coor_cur[1])**2)
    dis_next = math.sqrt((coor_gt[0]-coor_next[0])**2 + (coor_gt[1]-coor_next[1])**2)
    return dis_cur - dis_next


def right_action(coor_gt, coor_cur, radius_gt, radius):
    next_coordinate = coor_cur.copy()
    action = 0
    next_coordinate[0] -= radius/2.
    next_coordinate[1] += radius/2.
    area = IoW(coor_gt, next_coordinate, radius_gt, radius*0.7)

    next_coordinate = coor_cur.copy()
    next_coordinate[0] += radius/2.
    next_coordinate[1] -= radius/2.
    a = IoW(coor_gt, next_coordinate, radius_gt, radius*0.7)
    if area < a:
        action = 1
        area = a

    next_coordinate = coor_cur.copy()
    next_coordinate[0] -= radius/2.
    next_coordinate[1] -= radius/2.
    a = IoW(coor_gt, next_coordinate, radius_gt, radius*0.7)
    if area < a:
        action = 2
        area = a
    
    next_coordinate = coor_cur.copy()
    next_coordinate[0] += radius/2.
    next_coordinate[1] += radius/2.
    a = IoW(coor_gt, next_coordinate, radius_gt, radius*0.7)
    if area < a:
        action = 3
        area = a

    next_coordinate = coor_cur.copy()
    next_coordinate = coordinate
    a = IoW(coor_gt, next_coordinate, radius_gt, radius*.7)
    if area < a:
        action = 4
        area = a
    return action
    
    
    
class Net(nn.Module):
    def __init__(self, n_states, n_actions, n_hidden):
        super(Net, self).__init__()

        # 輸入層 (state) 到隱藏層，隱藏層到輸出層 (action)
        self.fc1 = nn.Linear(n_states, n_hidden)
        self.fc2 = nn.Linear(n_hidden, 32)
        self.out = nn.Linear(32, n_actions)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x) # ReLU # activation
        x = self.fc2(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value


class DQN(object):
    def __init__(self, n_states, n_actions, n_hidden, batch_size, lr, gamma, target_replace_iter, memory_capacity):
        self.eval_net, self.target_net = Net(n_states, n_actions, n_hidden), Net(n_states, n_actions, n_hidden)

        self.memory = np.zeros((memory_capacity, n_states * 2 + 2)) # 每個 memory 中的 experience 大小為 (state + next state + reward + action)
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=lr)
        self.loss_func = nn.MSELoss()
        self.memory_counter = 0
        self.learn_step_counter = 0 # 讓 target network 知道什麼時候要更新

        self.n_states = n_states
        self.n_actions = n_actions
        self.n_hidden = n_hidden
        self.batch_size = batch_size
        self.lr = lr
        self.gamma = gamma
        self.target_replace_iter = target_replace_iter
        self.memory_capacity = memory_capacity

    def choose_action(self, state, epsilon):
        x = torch.unsqueeze(torch.FloatTensor(state), 0)

        # epsilon-greedy
        if (np.random.uniform() < epsilon):
            action = np.random.randint(0, self.n_actions)
        else: 
            actions_value = self.eval_net(x) # 以現有 eval net 得出各個 action 的分數
            action = torch.max(actions_value, 1)[1].data.numpy()[0] # 挑選最高分的 action

        return action
    

    def store_transition(self, state, action, reward, next_state):
        # 打包 experience
        transition = np.hstack((state, [action, reward], next_state))

        # 存進 memory；舊 memory 可能會被覆蓋
        index = self.memory_counter % self.memory_capacity
        self.memory[index, :] = transition
        self.memory_counter += 1
        
    def BP(self, state, next_state, reward, right):
        q_eval = self.eval_net(torch.FloatTensor(state))
        q_target = torch.zeros([5,], dtype=torch.float32)
        q_target[right] = 1
        print("loss q", q_eval)
        print(q_target)
        loss = self.loss_func(q_eval, q_target)
#         q_eval = self.eval_net(torch.FloatTensor(state))
#         q_next = self.target_net(torch.FloatTensor(next_state)).detach()
#         q_target = reward + self.gamma * q_next
#         print("loss q", q_eval, q_target)
#         loss = self.loss_func(q_eval, q_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        
    def learn(self):
        # 隨機取樣 batch_size 個 experience
        sample_index = np.random.choice(self.memory_capacity, self.batch_size)
        b_memory = self.memory[sample_index, :]
        b_state = torch.FloatTensor(b_memory[:, :self.n_states])
        b_action = torch.LongTensor(b_memory[:, self.n_states:self.n_states+1].astype(int))
        b_reward = torch.FloatTensor(b_memory[:, self.n_states+1:self.n_states+2])
        b_next_state = torch.FloatTensor(b_memory[:, -self.n_states:])

        # 計算現有 eval net 和 target net 得出 Q value 的落差
        q_eval = self.eval_net(b_state).gather(1, b_action) # 重新計算這些 experience 當下 eval net 所得出的 Q value
        q_next = self.target_net(b_next_state).detach() # detach 才不會訓練到 target net
        q_target = b_reward + self.gamma * q_next.max(1)[0].view(self.batch_size, 1) # 計算這些 experience 當下 target net 所得出的 Q value
        loss = self.loss_func(q_eval, q_target)

        # Backpropagation
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # 每隔一段時間 (target_replace_iter), 更新 target net，即複製 eval net 到 target net
        self.learn_step_counter += 1
        if self.learn_step_counter % self.target_replace_iter == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
            

color = ['#F5B4B3', '#CA885B', '#DAE358', '#9DE358', '#58E39D', \
        '#58E3E1', '#58A2E3', '#5867E3', '#9D58E3', '#E158E3', '#E358B0', '#E35869']

In [ ]:
# for floor in range(0,5):
#     for building in range(0,3):
for floor in range(2,3):
    for building in range(0,1):
        
        ## Count the number of data points in building id & floor id
        data_num = 0
        with open("1478167720_9233432_trainingData.csv", newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for row in spamreader:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                data_num += 1
        print(data_num)
        ## if there are no data, continue to next floor 
        if (data_num == 0):
            continue
            
        ## Load data points in
        wifi_loc_time = np.zeros(shape = (data_num, 524))
        i=-1
        with open("1478167720_9233432_trainingData.csv", newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for row in spamreader:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                i = i+1
                if (i > data_num):
                    break
                # wifi
                wifi_loc_time[i-1][:520] = np.array(row[:520])
                # location x, y
                wifi_loc_time[i-1][520:522] = np.array(row[520:522])
                # userID
                wifi_loc_time[i-1][522] = np.array(row[526])
                # time stamp
                wifi_loc_time[i-1][-1] = np.array(row[-1])
        
        ## Sort by time stamp
        wifi_loc_time = wifi_loc_time[wifi_loc_time[:,-1].argsort()]
        
        ## Map boundaries
        longitude_list = np.array([max(wifi_loc_time[:, 520]), -1\
                                   , min(wifi_loc_time[:, 520])])
        latitude_list = np.array([max(wifi_loc_time[:, 521]), -1\
                                   , min(wifi_loc_time[:, 521])])
        
        ## KNN initial calculation
        nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(wifi_loc_time[:,:520])
        distances, indices = nbrs.kneighbors(wifi_loc_time[:,:520])

        ## DQN's hyper para
        n_actions = 5
        # state: RSSI (520), coordinate (2), radius (1), history (50)
        n_states = 520 + 2 + 1 + 50 
        n_hidden = 512
        batch_size = 100
        gamma = 0.1 # reward discount factor
        target_replace_iter = 100
        memory_capacity = 200
        n_episodes = 10000
        lr = 0.01
        eps = 0
        max_search_steps = 10
        it = 0
        cost_it = 0
        avg_it = 0
        delta = 0.7
        log_step = 500
        Rewards = 0
        radius_gt = 0.5
        it_list = []
        
        dqn = DQN(n_states, n_actions, n_hidden, batch_size, lr, gamma, target_replace_iter, memory_capacity)

        ## DQN training
        for k in range(n_episodes):
            print("Epoch - ", k, "eps : ", eps)
            avg_it = 0
            for i in range(len(wifi_loc_time)):
                # some important variables used for training
                Rewards = 0
                Goal = False
                alpha = 0.7
                cost_it = 0
                next_coordinate = np.array([0, 0])
                next_radius = 0
                rect1 = []
                
                ## 1. KNN locates initial coordinates and radius
                Hx = 0.
                Hy = 0.
                for m in range(3):
                    Hx += wifi_loc_time[indices[i, m], 520]
                    Hy += wifi_loc_time[indices[i, m], 521]
                Hx /= 3.
                Hy /= 3.
                coordinate = np.array([Hx, Hy])
                radius = 10.
                # initial history, 5n vector
                history = np.zeros(shape=(5*max_search_steps,), dtype=int)
                
                ## 2. Check initial KNN IoW
                while True:
                    IoW_cur = IoW(wifi_loc_time[i, 520:522], coordinate, radius_gt, radius)
                    if (IoW_cur == 0):
                        radius *= 1.5
                    elif (IoW_cur > delta):
                        print("Precise location!")
                        Goal = True
                        break
                    else:
                        break
                if (Goal == True):
                    continue
                # initial state: RSSI (520), coordinate (2), radius (1), history (50)
                state = np.concatenate((wifi_loc_time[i, :520], coordinate.copy(), np.array([radius]), history.copy()), axis=0)
                
                ## Plot gt region
                if k % log_step == 0:
                    fig = plt.figure()
                    ax = fig.add_subplot(111)
                    plt.xlim(longitude_list[0], longitude_list[2])
                    plt.ylim(latitude_list[0], latitude_list[2])
                    rect0 = plt.Rectangle((wifi_loc_time[i,520]-radius_gt, wifi_loc_time[i,521]-radius_gt), 2*radius_gt, 2*radius_gt, alpha=0.9)
                    rect1.append(plt.Rectangle((coordinate[0]-radius, coordinate[1]-radius), 2*radius, 2*radius, alpha = 0.6, color = color[-1]))
                
                ## 3. Searching starts
                for t in range(max_search_steps):
                    right = right_action(wifi_loc_time[i, 520:522], coordinate, radius_gt, radius)
                    print(t, "round, right action is ", right)
                    it = 0
                    if (radius < 0.5):
                        print("Radius is small enough and searching ends")
                        Goal = True
                        break
                    
                    
                    while True:
                        it += 1
                        ### (1) select an action
                        action = dqn.choose_action(state, eps)
                        print("[", action, "]", it)
                        ### (1) - 1. New Center
                        ### 0 -> "Up Left"
                        ### 1 -> "Up Right"
                        ### 2 -> "Down Left"
                        ### 3 -> "Down Right"
                        ### 4 -> "Center"
                        next_coordinate = coordinate.copy()
                        if (action == 0):
                            next_coordinate[0] -= radius/2.
                            next_coordinate[1] += radius/2.
                        elif (action == 1):
                            next_coordinate[0] += radius/2.
                            next_coordinate[1] -= radius/2.
                        elif (action == 2):
                            next_coordinate[0] -= radius/2.
                            next_coordinate[1] -= radius/2.
                        elif (action == 3):
                            next_coordinate[0] += radius/2.
                            next_coordinate[1] += radius/2.
                        else:
                            next_coordinate = coordinate
                        ### (1) - 2. New radius
                        next_radius = radius * alpha
                        ### (1) - 3. New IoW
                        next_IoW = IoW(wifi_loc_time[i, 520:522], next_coordinate, radius_gt, next_radius)
                        print("  IoW", IoW_cur, "->", next_IoW)
                        if (next_IoW > delta and action == right):
                            print("Precise location!")
                            Goal = True
                            cost_it += it
                            del next_coordinate
                            break
                        elif (next_IoW > IoW_cur and action == right):
                            # close score
                            reward = distance_progress(wifi_loc_time[i, 520:522], coordinate, next_coordinate)
                            print("  [C]  ontinue next round of searching, reward", reward, "\n   location", coordinate, "->", next_coordinate, "\n   radius", radius,"->" , next_radius)
                            IoW_cur = next_IoW
                            next_history = history.copy()
                            one_hot = t*5 + action
                            next_history[one_hot] = 1
                            next_state = np.concatenate((wifi_loc_time[i,:520], next_coordinate, np.array([next_radius]), next_history), axis=0)
                            dqn.store_transition(state.copy(), action, reward, next_state.copy())
                            radius = next_radius
                            coordinate = next_coordinate
                            history = next_history.copy()
                            state = next_state.copy()
                            Rewards += reward
                            del next_history
                            del next_state
                            del next_coordinate
                            cost_it += it
                            if k % log_step == 0:
                                # Plot
                                rect1.append(plt.Rectangle((coordinate[0]-radius, coordinate[1]-radius), 2*radius, 2*radius, alpha = 0.6, color = color[t]))
                            break
                        else:
                            reward = distance_progress(wifi_loc_time[i, 520:522], coordinate, next_coordinate)
                            next_history = history.copy()
                            one_hot = t*5 + action
                            next_history[one_hot] = 1
                            next_state = np.concatenate((wifi_loc_time[i,:520], next_coordinate, np.array([next_radius]), next_history), axis=0)
                            Rewards += reward
                            # back_propagation
                            dqn.BP(state, next_state, reward, right)
                            print("  [R]  epeat this round, reward", reward, "\n   location", coordinate, "->", next_coordinate, "\n   radius", radius,"->" , next_radius)
                        if it > 10:
                            print("--------------------------Training fail!---------------------------")
                            Goal = True
                            break
                    
                    if (Goal == True):
                        print("End searching for ", i ,", costs ", cost_it)
                        break
                del coordinate
                del state
                if k % log_step == 0:
                    for x in rect1:
                        ax.add_patch(x)
                    ax.add_patch(rect0)
                    plt.title("loc: "+str(i)+" epoch "+str(k)+" cost total searching "+str(cost_it)+" times")
                    plt.savefig("loc_"+str(i)+"_e_"+str(k))
                    plt.close()
                    plt.cla()
                    plt.clf()
                    rect1.clear()
                if dqn.memory_counter > memory_capacity:
                    dqn.learn()
                avg_it += cost_it
            eps *= 0.95
            clear_output(wait=True)
            it_list.append(avg_it/len(wifi_loc_time))
            print("avg_it : ", avg_it/len(wifi_loc_time))
            
                        

1443
Epoch -  0 eps :  0
0 round, right action is  0
[ 0 ] 1
  IoW 0.0025 -> 0.00510204081632653
  [C]  ontinue next round of searching, reward 1.7207294710458596 
   location [  -7645.51943382 4864921.38671716] -> [  -7650.51943382 4864926.38671716] 
   radius 10.0 -> 7.0
1 round, right action is  2
[ 0 ] 1
  IoW 0.00510204081632653 -> 0.0
loss q tensor([ -3308.7859, -19188.2539, -10671.3721, -32134.4375, -13987.9248],
       grad_fn=<AddBackward0>)
tensor([0., 0., 1., 0., 0.])
  [R]  epeat this round, reward -2.8344545496077176 
   location [  -7650.51943382 4864926.38671716] -> [  -7654.01943382 4864929.88671716] 
   radius 7.0 -> 4.8999999999999995
[ 3 ] 2
  IoW 0.00510204081632653 -> 0.0
loss q tensor([   477.5814, -19155.9160,  71650.2812, 242667.4219,  51937.7734],
       grad_fn=<AddBackward0>)
tensor([0., 0., 1., 0., 0.])
  [R]  epeat this round, reward -4.82248209552183 
   location [  -7650.51943382 4864926.38671716] -> [  -7647.01943382 4864929.88671716] 
   radius 7.0 -> 4

0 round, right action is  2
[ 0 ] 1
  IoW 0.0025 -> 0.0
loss q tensor([ 0.1419, -0.1147, -0.1982, -0.0758,  0.1147], grad_fn=<AddBackward0>)
tensor([0., 0., 1., 0., 0.])
  [R]  epeat this round, reward -1.4421051599045809 
   location [  -7646.98646715 4864924.64711716] -> [  -7651.98646715 4864929.64711716] 
   radius 10.0 -> 7.0
[ 0 ] 2
  IoW 0.0025 -> 0.0
loss q tensor([ 0.1417, -0.1145, -0.1984, -0.0760,  0.1144], grad_fn=<AddBackward0>)
tensor([0., 0., 1., 0., 0.])
  [R]  epeat this round, reward -1.4421051599045809 
   location [  -7646.98646715 4864924.64711716] -> [  -7651.98646715 4864929.64711716] 
   radius 10.0 -> 7.0
[ 0 ] 3
  IoW 0.0025 -> 0.0
loss q tensor([ 0.1415, -0.1144, -0.1987, -0.0762,  0.1142], grad_fn=<AddBackward0>)
tensor([0., 0., 1., 0., 0.])
  [R]  epeat this round, reward -1.4421051599045809 
   location [  -7646.98646715 4864924.64711716] -> [  -7651.98646715 4864929.64711716] 
   radius 10.0 -> 7.0
[ 0 ] 4
  IoW 0.0025 -> 0.0
loss q tensor([ 0.1413, -0.11

0 round, right action is  1
[ 0 ] 1
  IoW 0.0025 -> 0.0
loss q tensor([ 0.1395, -0.1132, -0.2006, -0.0783,  0.1118], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0.])
  [R]  epeat this round, reward -5.96199100634322 
   location [  -7653.36143382 4864928.5702505 ] -> [  -7658.36143382 4864933.5702505 ] 
   radius 10.0 -> 7.0
[ 0 ] 2
  IoW 0.0025 -> 0.0
loss q tensor([ 0.1395, -0.1132, -0.2006, -0.0783,  0.1118], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0.])
  [R]  epeat this round, reward -5.96199100634322 
   location [  -7653.36143382 4864928.5702505 ] -> [  -7658.36143382 4864933.5702505 ] 
   radius 10.0 -> 7.0
[ 0 ] 3
  IoW 0.0025 -> 0.0
loss q tensor([ 0.1395, -0.1132, -0.2006, -0.0783,  0.1118], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0.])
  [R]  epeat this round, reward -5.96199100634322 
   location [  -7653.36143382 4864928.5702505 ] -> [  -7658.36143382 4864933.5702505 ] 
   radius 10.0 -> 7.0
[ 0 ] 4
  IoW 0.0025 -> 0.0
loss q tensor([ 0.1395, -0.1132, -0

0 round, right action is  2
[ 0 ] 1
  IoW 0.0025 -> 0.0
loss q tensor([ 0.1394, -0.1131, -0.2007, -0.0784,  0.1117], grad_fn=<AddBackward0>)
tensor([0., 0., 1., 0., 0.])
  [R]  epeat this round, reward -3.097539723585774 
   location [  -7647.88816715 4864927.79288383] -> [  -7652.88816715 4864932.79288383] 
   radius 10.0 -> 7.0
[ 0 ] 2
  IoW 0.0025 -> 0.0
loss q tensor([ 0.1394, -0.1131, -0.2007, -0.0784,  0.1117], grad_fn=<AddBackward0>)
tensor([0., 0., 1., 0., 0.])
  [R]  epeat this round, reward -3.097539723585774 
   location [  -7647.88816715 4864927.79288383] -> [  -7652.88816715 4864932.79288383] 
   radius 10.0 -> 7.0
[ 0 ] 3
  IoW 0.0025 -> 0.0
loss q tensor([ 0.1394, -0.1131, -0.2007, -0.0784,  0.1117], grad_fn=<AddBackward0>)
tensor([0., 0., 1., 0., 0.])
  [R]  epeat this round, reward -3.097539723585774 
   location [  -7647.88816715 4864927.79288383] -> [  -7652.88816715 4864932.79288383] 
   radius 10.0 -> 7.0
[ 0 ] 4
  IoW 0.0025 -> 0.0
loss q tensor([ 0.1394, -0.1131,

0 round, right action is  3
[ 0 ] 1
  IoW 0.0025 -> 0.0
loss q tensor([ 0.1393, -0.1130, -0.2007, -0.0784,  0.1117], grad_fn=<AddBackward0>)
tensor([0., 0., 0., 1., 0.])
  [R]  epeat this round, reward -1.3818858269559762 
   location [  -7638.3346     4864957.64423333] -> [  -7643.3346     4864962.64423333] 
   radius 10.0 -> 7.0
[ 0 ] 2
  IoW 0.0025 -> 0.0
loss q tensor([ 0.1393, -0.1130, -0.2007, -0.0784,  0.1117], grad_fn=<AddBackward0>)
tensor([0., 0., 0., 1., 0.])
  [R]  epeat this round, reward -1.3818858269559762 
   location [  -7638.3346     4864957.64423333] -> [  -7643.3346     4864962.64423333] 
   radius 10.0 -> 7.0
[ 0 ] 3
  IoW 0.0025 -> 0.0
loss q tensor([ 0.1393, -0.1130, -0.2007, -0.0784,  0.1117], grad_fn=<AddBackward0>)
tensor([0., 0., 0., 1., 0.])
  [R]  epeat this round, reward -1.3818858269559762 
   location [  -7638.3346     4864957.64423333] -> [  -7643.3346     4864962.64423333] 
   radius 10.0 -> 7.0
[ 0 ] 4
  IoW 0.0025 -> 0.0
loss q tensor([ 0.1393, -0.11

...

  [R]  epeat this round, reward -7.0569249272074845 
   location [  -7639.01456667 4864909.3475    ] -> [  -7644.01456667 4864914.3475    ] 
   radius 10.0 -> 7.0
--------------------------Training fail!---------------------------
End searching for  486 , costs  0
0 round, right action is  0
[ 0 ] 1
  IoW 0.0025 -> 0.00510204081632653
  [C]  ontinue next round of searching, reward -5.871729011763421 
   location [  -7638.2135     4864906.56583333] -> [  -7643.2135     4864911.56583333] 
   radius 10.0 -> 7.0
1 round, right action is  1
[ 0 ] 1
  IoW 0.00510204081632653 -> 0.0
loss q tensor([ 6.1421,  0.1156, -0.1703, -0.0708,  0.0964], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0.])
  [R]  epeat this round, reward -4.913474864998412 
   location [  -7643.2135     4864911.56583333] -> [  -7646.7135     4864915.06583333] 
   radius 7.0 -> 4.8999999999999995
[ 0 ] 2
  IoW 0.00510204081632653 -> 0.0
loss q tensor([ 6.1419,  0.1157, -0.1703, -0.0708,  0.0964], grad_fn=<AddBackward0>)
t

0 round, right action is  1
[ 0 ] 1
  IoW 0.0025 -> 0.0
loss q tensor([ 6.1388,  0.1190, -0.1701, -0.0708,  0.0962], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0.])
  [R]  epeat this round, reward -6.938380161614627 
   location [  -7639.6383 4864911.5102] -> [  -7644.6383 4864916.5102] 
   radius 10.0 -> 7.0
[ 0 ] 2
  IoW 0.0025 -> 0.0
loss q tensor([ 6.1386,  0.1191, -0.1701, -0.0708,  0.0962], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0.])
  [R]  epeat this round, reward -6.938380161614627 
   location [  -7639.6383 4864911.5102] -> [  -7644.6383 4864916.5102] 
   radius 10.0 -> 7.0
[ 0 ] 3
  IoW 0.0025 -> 0.0
loss q tensor([ 6.1384,  0.1193, -0.1700, -0.0708,  0.0962], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0.])
  [R]  epeat this round, reward -6.938380161614627 
   location [  -7639.6383 4864911.5102] -> [  -7644.6383 4864916.5102] 
   radius 10.0 -> 7.0
[ 0 ] 4
  IoW 0.0025 -> 0.0
loss q tensor([ 6.1383,  0.1194, -0.1700, -0.0708,  0.0962], grad_fn=<AddBackwar

  [R]  epeat this round, reward -4.814672719217847 
   location [  -7642.2452 4864909.784 ] -> [  -7645.7452 4864913.284 ] 
   radius 7.0 -> 4.8999999999999995
[ 0 ] 11
  IoW 0.00510204081632653 -> 0.0
loss q tensor([ 6.1340,  0.1238, -0.1697, -0.0707,  0.0959], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0.])
  [R]  epeat this round, reward -4.814672719217847 
   location [  -7642.2452 4864909.784 ] -> [  -7645.7452 4864913.284 ] 
   radius 7.0 -> 4.8999999999999995
--------------------------Training fail!---------------------------
End searching for  491 , costs  1
0 round, right action is  1
[ 0 ] 1
  IoW 0.0025 -> 0.0
loss q tensor([ 6.1338,  0.1241, -0.1697, -0.0707,  0.0959], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0.])
  [R]  epeat this round, reward -6.784624910935431 
   location [  -7640.70626667 4864915.00066667] -> [  -7645.70626667 4864920.00066667] 
   radius 10.0 -> 7.0
[ 0 ] 2
  IoW 0.0025 -> 0.0
loss q tensor([ 6.1336,  0.1242, -0.1696, -0.0707,  0.0959], g

loss q tensor([ 6.1292,  0.1287, -0.1693, -0.0707,  0.0957], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0.])
  [R]  epeat this round, reward -4.945714795291043 
   location [  -7642.42686667 4864910.41726667] -> [  -7645.92686667 4864913.91726667] 
   radius 7.0 -> 4.8999999999999995
[ 0 ] 10
  IoW 0.00510204081632653 -> 0.0
loss q tensor([ 6.1291,  0.1289, -0.1693, -0.0707,  0.0957], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0.])
  [R]  epeat this round, reward -4.945714795291043 
   location [  -7642.42686667 4864910.41726667] -> [  -7645.92686667 4864913.91726667] 
   radius 7.0 -> 4.8999999999999995
[ 0 ] 11
  IoW 0.00510204081632653 -> 0.0
loss q tensor([ 6.1290,  0.1290, -0.1693, -0.0707,  0.0956], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0.])
  [R]  epeat this round, reward -4.945714795291043 
   location [  -7642.42686667 4864910.41726667] -> [  -7645.92686667 4864913.91726667] 
   radius 7.0 -> 4.8999999999999995
--------------------------Training fail!------

0 round, right action is  1
[ 0 ] 1
  IoW 0.0025 -> 0.0
loss q tensor([ 6.1253,  0.1328, -0.1690, -0.0706,  0.0954], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0.])
  [R]  epeat this round, reward -6.718407809308852 
   location [  -7640.69886667 4864914.975     ] -> [  -7645.69886667 4864919.975     ] 
   radius 10.0 -> 7.0
[ 0 ] 2
  IoW 0.0025 -> 0.0
loss q tensor([ 6.1252,  0.1329, -0.1690, -0.0706,  0.0954], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0.])
  [R]  epeat this round, reward -6.718407809308852 
   location [  -7640.69886667 4864914.975     ] -> [  -7645.69886667 4864919.975     ] 
   radius 10.0 -> 7.0
[ 0 ] 3
  IoW 0.0025 -> 0.0
loss q tensor([ 6.1250,  0.1331, -0.1689, -0.0706,  0.0954], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0.])
  [R]  epeat this round, reward -6.718407809308852 
   location [  -7640.69886667 4864914.975     ] -> [  -7645.69886667 4864919.975     ] 
   radius 10.0 -> 7.0
[ 0 ] 4
  IoW 0.0025 -> 0.0
loss q tensor([ 6.1248,  0.1332,